In [ ]:
#export
from local.imports import *
from local.test import *
from local.core import *
from local.layers import *
from local.data.pipeline import *
from local.data.source import *
from local.data.core import *
from local.data.external import *
from local.notebook.showdoc import show_doc
from local.optimizer import *
from local.learner import *

In [ ]:
#default_exp callback.hook

# Model hooks

> Callback and helper function to add hooks in models

In [ ]:
from local.utils.test import *

## What are hooks?

Hooks are function you can attach to a particular layer in your model and that will be executed in the foward pass (for forward hooks) or backward pass (for backward hooks). Here we begin with an introduction around hooks, but you should jump to `HookCallback` if you quickly want to implemet one (and read the following example `ActivationStats`).

Forward hooks are functions that take three arguments, the layer it's applied to, the input of that layer and the output of that layer.

In [ ]:
tst_model = nn.Linear(5,3)
def example_forward_hook(m,i,o): print(m,i,o)
    
x = torch.randn(4,5)
hook = tst_model.register_forward_hook(example_forward_hook)
y = tst_model(x)
hook.remove()

Linear(in_features=5, out_features=3, bias=True) (tensor([[-0.2044, -0.2237, -0.4806, -1.3557, -1.5280],
        [-1.3220,  0.2757, -0.0922,  2.0723, -0.3677],
        [ 1.5474, -0.0833,  0.2422, -1.2864, -0.4185],
        [-0.4262, -0.0400,  1.0578,  0.3906, -0.6471]]),) tensor([[-0.4217, -0.9593, -0.0705],
        [ 1.2808,  0.1823, -0.1422],
        [-0.8413, -0.2547, -0.4600],
        [ 0.2718,  0.0679, -0.2339]], grad_fn=<AddmmBackward>)


Backward hooks are functions that take three arguments: the layer it's applied to, the gradients of the loss with respect to the input, and the gradients with respect to the output.

In [ ]:
def example_backward_hook(m,gi,go): print(m,gi,go)
hook = tst_model.register_backward_hook(example_backward_hook)

x = torch.randn(4,5)
y = tst_model(x)
loss = y.pow(2).mean()
loss.backward()
hook.remove()

Linear(in_features=5, out_features=3, bias=True) (tensor([ 0.0507,  0.1921, -0.1170]), None, tensor([[-0.1473, -0.0476, -0.1806],
        [-0.1710, -0.4402, -0.3973],
        [-0.1518,  0.1404, -0.0820],
        [ 0.3362,  0.2431,  0.1652],
        [ 0.0226, -0.0616, -0.0232]])) (tensor([[ 0.0861, -0.0144, -0.0898],
        [ 0.0267,  0.1088,  0.0757],
        [-0.1316, -0.0400, -0.1311],
        [ 0.0695,  0.1378,  0.0282]]),)


Hooks can change the input/output of a layer, or the gradients, print values or shapes. If you want to store something related to theses inputs/outputs, it's best to have you hook associated to a class so that it can put it in the state of an instance of that class.

## Hook -

In [ ]:
#export
@docs
class Hook():
    "Create a hook on `m` with `hook_func`."
    def __init__(self, m, hook_func, is_forward=True, detach=True, cpu=False):
        self.hook_func,self.detach,self.cpu,self.stored = hook_func,detach,cpu,None
        f = m.register_forward_hook if is_forward else m.register_backward_hook
        self.hook = f(self.hook_fn)
        self.removed = False

    def hook_fn(self, module, input, output):
        "Applies `hook_func` to `module`, `input`, `output`."
        if self.detach: input,output = to_detach(input, cpu=self.cpu),to_detach(output, cpu=self.cpu)
        self.stored = self.hook_func(module, input, output)

    def remove(self):
        "Remove the hook from the model."
        if not self.removed:
            self.hook.remove()
            self.removed=True

    def __enter__(self, *args): return self
    def __exit__(self, *args): self.remove()
        
    _docs = dict(__enter__="Register the hook",
                 __exit__="Remove the hook")

This will be called during the forward pass if `is_forward=True`, the backward pass otherwise, and will optionally `detach` and put on the `cpu` the (gradient of the) input/output of the model before passing them to `hook_func`. The result of `hook_func` will be stored in the `stored` attribute of the `Hook`.

In [ ]:
tst_model = nn.Linear(5,3)
hook = Hook(tst_model, lambda m,i,o: o)
y = tst_model(x)
test_eq(hook.stored, y)

In [ ]:
show_doc(Hook.hook_fn)

<h4 id="<code>Hook.hook_fn</code>" class="doc_header"><code>Hook.hook_fn</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/14_callback_hook.ipynb#Hook--" class="source_link" style="float:right">[source]</a></h4>

> <code>Hook.hook_fn</code>(**`module`**, **`input`**, **`output`**)

Applies `hook_func` to `module`, `input`, `output`.

In [ ]:
show_doc(Hook.remove)

<h4 id="<code>Hook.remove</code>" class="doc_header"><code>Hook.remove</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/14_callback_hook.ipynb#Hook--" class="source_link" style="float:right">[source]</a></h4>

> <code>Hook.remove</code>()

Remove the hook from the model.

> Note: It's important to properly remove your hooks for your model when you're done to avoid them being called again next time your model is applied to some inputs, and to free the memory that go with their state.

In [ ]:
tst_model = nn.Linear(5,10)
x = torch.randn(4,5)
y = tst_model(x)
hook = Hook(tst_model, example_forward_hook)
test_stdout(lambda: tst_model(x), f"{tst_model} [{x}] {y.detach()}")
hook.remove()
test_stdout(lambda: tst_model(x), "")

### Context Manager

Since it's very important to remove your `Hook` even if your code is interrupted by some bug, `Hook` can be used as context managers.

In [ ]:
show_doc(Hook.__enter__)

<h4 id="<code>Hook.__enter__</code>" class="doc_header"><code>Hook.__enter__</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/14_callback_hook.ipynb#Hook--" class="source_link" style="float:right">[source]</a></h4>

> <code>Hook.__enter__</code>(**\*`args`**)

Register the hook

In [ ]:
show_doc(Hook.__exit__)

<h4 id="<code>Hook.__exit__</code>" class="doc_header"><code>Hook.__exit__</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/14_callback_hook.ipynb#Hook--" class="source_link" style="float:right">[source]</a></h4>

> <code>Hook.__exit__</code>(**\*`args`**)

Remove the hook

In [ ]:
tst_model = nn.Linear(5,10)
x = torch.randn(4,5)
y = tst_model(x)
with Hook(tst_model, example_forward_hook) as h:
    test_stdout(lambda: tst_model(x), f"{tst_model} [{x}] {y.detach()}")
test_stdout(lambda: tst_model(x), "")

In [ ]:
#export
def _hook_inner(m,i,o): return o if isinstance(o,Tensor) or is_listy(o) else list(o)

def hook_output(module, detach=True, cpu=False, grad=False):
    "Return a `Hook` that stores activations of `module` in `self.stored`"
    return Hook(module, _hook_inner, detach=detach, cpu=cpu, is_forward=not grad)

The activations stored are the gradients if `grad=True`, otherwise the output of `module`. If `detach=True` they are detached from their history, and if `cpu=True`, they're put on the CPU.

In [ ]:
tst_model = nn.Linear(5,10)
x = torch.randn(4,5)
with hook_output(tst_model) as h:
    y = tst_model(x)
    test_eq(y, h.stored)
    assert not h.stored.requires_grad
    
with hook_output(tst_model, grad=True) as h:
    y = tst_model(x)
    loss = y.pow(2).mean()
    loss.backward()
    test_close(2*y / y.numel(), h.stored[0])

In [ ]:
#cuda
with hook_output(tst_model, cpu=True) as h:
    y = tst_model.cuda()(x.cuda())
    test_eq(h.stored.device, torch.device('cpu'))

## Hooks -

In [ ]:
#export
@docs
class Hooks():
    "Create several hooks on the modules in `ms` with `hook_func`."
    def __init__(self, ms, hook_func, is_forward=True, detach=True, cpu=False):
        self.hooks = [Hook(m, hook_func, is_forward, detach, cpu) for m in ms]

    def __getitem__(self,i): return self.hooks[i]
    def __len__(self):       return len(self.hooks)
    def __iter__(self):      return iter(self.hooks)
    @property
    def stored(self):        return [o.stored for o in self]

    def remove(self):
        "Remove the hooks from the model."
        for h in self.hooks: h.remove()

    def __enter__(self, *args): return self
    def __exit__ (self, *args): self.remove()
            
    _docs = dict(stored = "The states saved in each hook.",
                 __enter__="Register the hooks",
                 __exit__="Remove the hooks")

In [ ]:
layers = [nn.Linear(5,10), nn.ReLU(), nn.Linear(10,3)]
tst_model = nn.Sequential(*layers)
hooks = Hooks(tst_model, lambda m,i,o: o)
y = tst_model(x)
test_eq(hooks.stored[0], layers[0](x))
test_eq(hooks.stored[1], F.relu(layers[0](x)))
test_eq(hooks.stored[2], y)
hooks.remove()

In [ ]:
show_doc(Hooks.stored, name='Hooks.stored')

<h4 id="<code>Hooks.stored</code>" class="doc_header"><code>Hooks.stored</code><a href="" class="source_link" style="float:right">[source]</a></h4>

The states saved in each hook.

In [ ]:
show_doc(Hooks.remove)

<h4 id="<code>Hooks.remove</code>" class="doc_header"><code>Hooks.remove</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/14_callback_hook.ipynb#Hooks--" class="source_link" style="float:right">[source]</a></h4>

> <code>Hooks.remove</code>()

Remove the hooks from the model.

### Context Manager

Like `Hook` , you can use `Hooks` as context managers.

In [ ]:
show_doc(Hooks.__enter__)

<h4 id="<code>Hooks.__enter__</code>" class="doc_header"><code>Hooks.__enter__</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/14_callback_hook.ipynb#Hooks--" class="source_link" style="float:right">[source]</a></h4>

> <code>Hooks.__enter__</code>(**\*`args`**)

Register the hooks

In [ ]:
show_doc(Hooks.__exit__)

<h4 id="<code>Hooks.__exit__</code>" class="doc_header"><code>Hooks.__exit__</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/14_callback_hook.ipynb#Hooks--" class="source_link" style="float:right">[source]</a></h4>

> <code>Hooks.__exit__</code>(**\*`args`**)

Remove the hooks

In [ ]:
layers = [nn.Linear(5,10), nn.ReLU(), nn.Linear(10,3)]
tst_model = nn.Sequential(*layers)
with Hooks(layers, lambda m,i,o: o) as h:
    y = tst_model(x)
    test_eq(h.stored[0], layers[0](x))
    test_eq(h.stored[1], F.relu(layers[0](x)))
    test_eq(h.stored[2], y)

In [ ]:
#export
def hook_outputs(modules, detach=True, cpu=False, grad=False)->Hooks:
    "Return `Hooks` that store activations of all `modules` in `self.stored`"
    return Hooks(modules, _hook_inner, detach=detach, cpu=cpu, is_forward=not grad)

The activations stored are the gradients if `grad=True`, otherwise the output of `modules`. If `detach=True` they are detached from their history, and if `cpu=True`, they're put on the CPU.

In [ ]:
layers = [nn.Linear(5,10), nn.ReLU(), nn.Linear(10,3)]
tst_model = nn.Sequential(*layers)
x = torch.randn(4,5)
with hook_outputs(layers) as h:
    y = tst_model(x)
    test_eq(h.stored[0], layers[0](x))
    test_eq(h.stored[1], F.relu(layers[0](x)))
    test_eq(h.stored[2], y)
    for s in h.stored: assert not s.requires_grad
    
with hook_outputs(layers, grad=True) as h:
    y = tst_model(x)
    loss = y.pow(2).mean()
    loss.backward()
    g = 2*y / y.numel()
    test_close(g, h.stored[2][0])
    g = g @ layers[2].weight.data
    test_close(g, h.stored[1][0])
    g = g * (layers[0](x) > 0).float()
    test_close(g, h.stored[0][0])

In [ ]:
#cuda
with hook_outputs(tst_model, cpu=True) as h:
    y = tst_model.cuda()(x.cuda())
    for s in h.stored: test_eq(s.device, torch.device('cpu'))

## HookCallback -

To make hooks easy to use, we wrapped a version in a Callback where you just have to implement a `hook` function (plus any element you might need).

In [ ]:
#export
def has_params(m):
    "Check if `m` has at least one parameter"
    return len(list(m.parameters())) > 0

In [ ]:
assert has_params(nn.Linear(3,4))
assert has_params(nn.LSTM(4,5,2))
assert not has_params(nn.ReLU())

In [ ]:
#export
class HookCallback(Callback):
    "`Callback` that can be used to register hooks on `modules`"
    def __init__(self, hook=None, modules=None, do_remove=True, is_forward=True, detach=True, cpu=False):
        self.modules,self.do_remove = modules,do_remove
        self.is_forward,self.detach,self.cpu = is_forward,detach,cpu
        if hook is not None: setattr(self, 'hook', hook)
    
    def begin_fit(self):
        "Register the `Hooks` on `self.modules`."
        if not self.modules:
            self.modules = [m for m in flatten_model(self.model) if has_params(m)]
        self.hooks = Hooks(self.modules, self.hook, self.is_forward, self.detach, self.cpu)

    def after_fit(self):
        "Remove the `Hooks`."
        if self.do_remove: self._remove()

    def _remove(self): 
        if getattr(self, 'hooks', None): self.hooks.remove()
    
    def __del__(self): self._remove()

You can either subclass and implement a `hook` function (along with any event you want) or pass that a `hook` function when initializing. Such a function needs to take three argument: a layer, input and output (for a backward hook, input means gradient with respect to the inputs, output, gradient with respect to the output) and can either modify them or update the state according to them.

If not provided, `modules` will default to the layers of `self.model` that have a `weight` attribute. Depending on `do_remove`, the hooks will be properly removed at the end of training (or in case of error). `is_forward` , `detach` and `cpu` are passed to `Hooks`.

The function called at each forward (or backward) pass is `self.hook` and must be implemented when subclassing this callback.

In [ ]:
class TstCallback(HookCallback):
    def hook(self, m, i, o): return o
    def after_batch(self): test_eq(self.hooks.stored[0], self.pred)
        
learn = synth_learner(n_trn=5, cbs = TstCallback())
learn.fit(1)

[0, 8.967827796936035, 9.353962898254395, '00:00']


In [ ]:
class TstCallback(HookCallback):
    def __init__(self, modules=None, do_remove=True, detach=True, cpu=False):
        super().__init__(None, modules, do_remove, False, detach, cpu)
    def hook(self, m, i, o): return o
    def after_batch(self):
        if self.training:
            test_eq(self.hooks.stored[0][0], 2*(self.pred-self.yb)/self.pred.shape[0])
        
learn = synth_learner(n_trn=5, cbs = TstCallback())
learn.fit(1)

[0, 5.6319499015808105, 3.702653408050537, '00:00']


In [ ]:
show_doc(HookCallback.begin_fit)

<h4 id="<code>HookCallback.begin_fit</code>" class="doc_header"><code>HookCallback.begin_fit</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/14_callback_hook.ipynb#HookCallback--" class="source_link" style="float:right">[source]</a></h4>

> <code>HookCallback.begin_fit</code>()

Register the [`Hooks`](/callback.hook.html#Hooks) on `self.modules`.

In [ ]:
show_doc(HookCallback.after_fit)

<h4 id="<code>HookCallback.after_fit</code>" class="doc_header"><code>HookCallback.after_fit</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/14_callback_hook.ipynb#HookCallback--" class="source_link" style="float:right">[source]</a></h4>

> <code>HookCallback.after_fit</code>()

Remove the [`Hooks`](/callback.hook.html#Hooks).

An example of such a `HookCallback` is the following, that stores the mean and stds of activations that go through the network.

In [ ]:
#exports
@docs
class ActivationStats(HookCallback):
    "Callback that record the mean and std of activations."

    def begin_fit(self):
        "Initialize stats."
        super().begin_fit()
        self.stats = []

    def hook(self, m, i, o): return o.mean().item(),o.std().item()
    
    def after_batch(self):
        "Take the stored results and puts it in `self.stats`"
        if self.training: self.stats.append(self.hooks.stored)
    
    def after_fit(self):
        "Polish the final result."
        self.stats = tensor(self.stats).permute(2,1,0)
        super().after_fit()
        
    _docs = dict(hook="Take the mean and std of the output")

In [ ]:
learn = synth_learner(n_trn=5, cbs = ActivationStats())
learn.fit(1)

[0, 15.714694023132324, 6.2435173988342285, '00:00']


In [ ]:
learn.activation_stats.stats

tensor([[[0.3231, 0.2611, 0.4024, 0.8207, 0.6421]],

        [[0.8159, 1.1559, 1.2019, 0.8848, 0.3619]]])

The first line contains the means of the outputs of the model for each batch in the training set, the second line their standard deviations.

In [ ]:
#hide
class TstCallback(HookCallback):
    def hook(self, m, i, o): return o
    def begin_fit(self):
        super().begin_fit()
        self.means,self.stds = [],[]
    
    def after_batch(self):
        if self.training:
            self.means.append(self.hooks.stored[0].mean().item())
            self.stds.append (self.hooks.stored[0].std() .item())

learn = synth_learner(n_trn=5, cbs = [TstCallback(), ActivationStats()])
learn.fit(1)
test_eq(learn.activation_stats.stats[0].squeeze(), tensor(learn.tst.means))
test_eq(learn.activation_stats.stats[1].squeeze(), tensor(learn.tst.stds))

[0, 29.84511375427246, 19.519798278808594, '00:00']


## Model summary

In [ ]:
#export
def total_params(m):
    "Give the number of parameters of a module and if it's trainable or not"
    params = sum([p.numel() for p in m.parameters()])
    trains = [p.requires_grad for p in m.parameters()]
    return params, (False if len(trains)==0 else trains[0])

In [ ]:
test_eq(total_params(nn.Linear(10,32)), (32*10+32,True))
test_eq(total_params(nn.Linear(10,32, bias=False)), (32*10,True))
test_eq(total_params(nn.BatchNorm2d(20)), (20*2, True))
test_eq(total_params(nn.BatchNorm2d(20, affine=False)), (0,False))
test_eq(total_params(nn.Conv2d(16, 32, 3)), (16*32*3*3 + 32, True))
test_eq(total_params(nn.Conv2d(16, 32, 3, bias=False)), (16*32*3*3, True))
#First ih layer 20->10, all else 10->10. *4 for the four gates
test_eq(total_params(nn.LSTM(20, 10, 2)), (4 * (20*10 + 10) + 3 * 4 * (10*10 + 10), True))

In [ ]:
#export
def layer_info(learn):
    def _track(m, i, o): 
        return (m.__class__.__name__,)+total_params(m)+(apply(lambda x:x.shape, o),)
    layers = [m for m in flatten_model(learn.model)]
    xb,_ = learn.data.train_dl.one_batch()
    with Hooks(layers, _track) as h:
        _ = learn.model.eval()(apply(lambda o:o[:1], xb))
        return h.stored

In [ ]:
m = nn.Sequential(nn.Linear(1,50), nn.ReLU(), nn.BatchNorm1d(50), nn.Linear(50, 1))
learn = synth_learner()
learn.model=m

In [ ]:
test_eq(layer_info(learn), [
    ('Linear', 100, True, [1, 50]),
    ('ReLU', 0, False, [1, 50]),
    ('BatchNorm1d', 100, True, [1, 50]),
    ('Linear', 51, True, [1, 1])
])

In [ ]:
#export core
class PrettyString(str):
    "Little hack to get strings to show properly in Jupyter."
    def __repr__(self): return self

In [ ]:
#export
def _print_shapes(o, bs):
    if isinstance(o, torch.Size): return ' x '.join([str(bs)] + [str(t) for t in o[1:]])
    else: return [_print_shapes(x, bs) for x in o]

In [ ]:
#export
@patch
def summary(self:Learner):
    "Print a summary of the model, optimizer and loss function."
    infos = layer_info(self)
    xb,_ = self.data.train_dl.one_batch()
    n,bs = 63,find_bs(xb)
    inp_sz = _print_shapes(apply(lambda x:x.shape, xb), bs)
    res = f"{self.model.__class__.__name__} (Input shape: {inp_sz})\n"
    res += "=" * n + "\n"
    res += f"{'Layer (type)':<20} {'Output Shape':<20} {'Param #':<10} {'Trainable':<10}\n"
    res += "=" * n + "\n"
    ps,trn_ps = 0,0
    for typ,np,trn,sz in infos:
        if sz is None: continue
        ps += np
        if trn: trn_ps += np
        res += f"{typ:<20} {_print_shapes(sz, bs):<20} {np:<10,} {str(trn):<10}\n"
        res += "_" * n + "\n"
    res += f"\nTotal params: {ps:,}\n"
    res += f"Total trainable params: {trn_ps:,}\n"
    res += f"Total non-trainable params: {ps - trn_ps:,}\n\n"
    res += f"Optimizer used: {self.opt_func}\nLoss function: {self.loss_func}\n\nCallbacks:\n"
    res += '\n'.join(f"  - {cb}" for cb in sort_by_run(self.cbs))
    return PrettyString(res)

In [ ]:
m = nn.Sequential(nn.Linear(1,50), nn.ReLU(), nn.BatchNorm1d(50), nn.Linear(50, 1))
for p in m[0].parameters(): p.requires_grad_(False)
learn = synth_learner()
learn.model=m
learn.summary()

Sequential (Input shape: 16 x 1)
Layer (type)         Output Shape         Param #    Trainable 
Linear               16 x 50              100        False     
_______________________________________________________________
ReLU                 16 x 50              0          False     
_______________________________________________________________
BatchNorm1d          16 x 50              100        True      
_______________________________________________________________
Linear               16 x 1               51         True      
_______________________________________________________________

Total params: 251
Total trainable params: 151
Total non-trainable params: 100

Optimizer used: functools.partial(<function SGD at 0x7f780a18f510>, mom=0.9)
Loss function: FlattenedLoss of MSELoss()

Callbacks:
  - TrainEvalCallback
  - Recorder

## Export -

In [ ]:
#hide
from local.notebook.export import notebook2script
notebook2script(all_fs=True)

Converted 00_test.ipynb.
Converted 01_core.ipynb.
Converted 02_data_pipeline.ipynb.
Converted 03_data_external.ipynb.
Converted 04_data_core.ipynb.
Converted 05_data_source.ipynb.
Converted 06_vision_core.ipynb.
Converted 07_pets_tutorial.ipynb.
Converted 08_augmentation.ipynb.
Converted 10_layers.ipynb.
Converted 11_optimizer.ipynb.
Converted 12_learner.ipynb.
Converted 13_callback_schedule.ipynb.
Converted 14_callback_hook.ipynb.
Converted 15_callback_progress.ipynb.
Converted 90_notebook_core.ipynb.
Converted 91_notebook_export.ipynb.
Converted 92_notebook_showdoc.ipynb.
Converted 93_notebook_export2html.ipynb.
Converted 94_index.ipynb.
Converted 95_synth_learner.ipynb.
